In [1]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, cohen_kappa_score, classification_report, confusion_matrix
from scipy.stats import rankdata
from numpy import linalg as LA
# import intel_extension_for_pytorch as ipex

%load_ext autotime

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import numpy as np
import torch.nn.functional as F

# from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import MultiStepLR
import time
import os
plt.style.use('ggplot')
device = 'cuda'
import os
import time
import math
from tqdm import tqdm
import logging
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, MultiStepLR

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

time: 172 ms (started: 2024-05-06 06:46:56 +06:00)


In [2]:
##################################################################
#   Utils
##################################################################


"""
Tensorboard logger code referenced from:
https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/04-utils/
Other helper functions:
https://github.com/cs230-stanford/cs230-stanford.github.io
"""

import json
import logging
import os
import shutil
import torch
from collections import OrderedDict
from torch.optim.lr_scheduler import _LRScheduler
# import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Params():
    """Class that loads hyperparameters from a json file.

    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)
            
    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']"""
        return self.__dict__


class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum/self.count

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.

    Example:
    ```
    logging.info("Starting training...")
    ```

    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


def save_checkpoint(state, is_best, checkpoint, epoch_checkpoint = False):
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth.tar')
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))
    if epoch_checkpoint == True:
        epoch_file = str(state['epoch']-1) + '.pth.tar'
        shutil.copyfile(filepath, os.path.join(checkpoint, epoch_file))





def load_checkpoint(checkpoint, model, optimizer=None):
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    try:

      if not os.path.exists(checkpoint):
        raise FileNotFoundError
    except FileNotFoundError:
      ("File doesn't exist {}".format(checkpoint))

    if torch.cuda.is_available():
        checkpoint = torch.load(checkpoint)
    else:
        # this helps avoid errors when loading single-GPU-trained weights onto CPU-model
        checkpoint = torch.load(checkpoint, map_location=lambda storage, loc: storage)

    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint


class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """

    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

time: 0 ns (started: 2024-05-06 06:46:56 +06:00)


In [3]:
transform = {
        'train': torchvision.transforms.Compose([
            torchvision.transforms.Resize([64,64]), # Resizing the image as the VGG only take 224 x 244 as input size
            torchvision.transforms.RandomHorizontalFlip(), # Flip the data horizontally
            torchvision.transforms.RandomVerticalFlip(), # Flip the data horizontally
#             torchvision.transforms.CenterCrop(64),
            torchvision.transforms.ColorJitter(),
#                         torchvision.transforms.GaussianBlur(),
            torchvision.transforms.RandAugment(),
#             torchvision.transforms.AugMix(),
            #TODO if it is needed, add the random crop
#             torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ]),
        'test': torchvision.transforms.Compose([
            torchvision.transforms.Resize([64,64]),
#             transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ])
    }

time: 32 ms (started: 2024-05-06 06:46:56 +06:00)


In [4]:
train_data_dir = '.\\train_images'
# train_coco = 'images_thermal_train/coco.json'
val_train_dir = '.\\val_images'
# val_coco = 'images_thermal_val/coco.json'
batch_size = 128

train_data = torchvision.datasets.ImageFolder(root=train_data_dir, transform=transform['train'])
train_dataloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

val_data = torchvision.datasets.ImageFolder(root=val_train_dir, transform=transform['test'])
val_dataloader = torch.utils.data.DataLoader(dataset=val_data, batch_size=64, shuffle=True)

time: 1.03 s (started: 2024-05-06 06:46:56 +06:00)


In [5]:
class_dictionary = {1: 'person', 2: 'bike', 3: 'car', 4: 'motor', 6: 'bus',  7: 'train', 8: 'truck', 10: 'light', 
                   11: 'hydrant', 12: 'sign', 17: 'dog', 37: 'skateboard', 73: 'stroller', 77: 'scooter', 79: 'other vehicle',
                   80: 'negative'}

time: 0 ns (started: 2024-05-06 06:46:57 +06:00)


In [6]:
# i =1
# s = list(class_dictionary)
# img= train_dataset[i][0]
# bbox = train_dataset[i][1]['boxes']
# classes = train_dataset[i][1]['labels']
# # print(len(classes))
# k = 0
# for i in val_dataset:
#     img= i[0]
#     bbox = i[1]['boxes']
#     classes = i[1]['labels']
#     for a,b in zip(bbox, classes):
# #         print(int(b))
#         xmin, ymin, xmax, ymax = a
#         pt1 = (int(xmin), int(ymin))
#         pt2 = (int(xmax), int(ymax))
#         crop = img.permute(1, 2, 0).numpy()
#         crop = crop[int(ymin):int(ymax), int(xmin):int(xmax)]
#         # vool = b not in s
#         # if b:
#         #     b = 'other'
#         cv2.imwrite(".\\val_images\\{0}\\crop_{1}.png".format(b,a), 255*crop)
#     # crop = img[0][int(ymin):int(ymax), int(xmin):int(xmax)]
#     # plt.grid(False)
#     # plt.axis('off')
#     # plt.imshow(crop, cmap = 'gray')
#     # plt.savefig('foo'+str(k)+'.png', dpi = 20)
#     # k+=1    
#     # plt.show()

#     # bnd_img = cv2.rectangle(img.permute(1, 2, 0).numpy(),pt1, pt2,(0,0,0),1)
#     # bnd_img = cv2.putText(
#     #     bnd_img,
#     #     str(j),
#     #     (int(xmin), int(ymin) - 10),
#     #     fontFace = cv2.FONT_HERSHEY_SIMPLEX,
#     #     fontScale = 0.3,
#     #     color = (0, 255, 255),
#     #     thickness=1)
    
#     # plt.imshow(bnd_img, cmap = 'gray')

time: 16 ms (started: 2024-05-06 06:46:57 +06:00)


In [7]:
train_data[0][0].shape

torch.Size([3, 64, 64])

time: 16 ms (started: 2024-05-06 06:46:57 +06:00)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

time: 0 ns (started: 2024-05-06 06:46:57 +06:00)


In [9]:
def loss_kd_regularization(outputs, labels):
    """
    loss function for mannually-designed regularization: Tf-KD_{reg}
    """
    alpha = 0.95
    T = 20
    correct_prob = 0.99    # the probability for correct class in u(k)
    loss_CE = F.cross_entropy(outputs, labels)
#     loss_CE = FocalLoss(gamma=0)(outputs,labels)
    K = outputs.size(1)

    teacher_soft = torch.ones_like(outputs).to(device)
    teacher_soft = teacher_soft*(1-correct_prob)/(K-1)  # p^d(k)
    for i in range(outputs.shape[0]):
        teacher_soft[i, labels[i]] = correct_prob
    loss_soft_regu = torch.nn.KLDivLoss()(F.log_softmax(outputs, dim=1),
                                    F.softmax(teacher_soft/T, dim=1))*1

    KD_loss = (1. - alpha)*loss_CE + alpha*loss_soft_regu

    return KD_loss

time: 0 ns (started: 2024-05-06 06:46:57 +06:00)


In [10]:
def evaluate_kd(model, dataloader, device = 'cuda', crivice = 1):
    """Evaluate the model on `num_steps` batches.
    
    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()
    total = 0
    correct = 0
    data_sc = []
    data_rc = []
    # compute metrics over the dataset
    for i, (data_batch, labels_batch) in enumerate(dataloader):
        # move to GPU if available
        data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)

        # compute model output
        output_batch = model(data_batch)
        
        # loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch, params)
        loss = 0.0  # force validation loss to zero to reduce computation time
        _, predicted = output_batch.max(1)
        total += labels_batch.size(0)
        correct += predicted.eq(labels_batch).sum().item()
        predicted = predicted.cpu().data.numpy()
        labels_batch = labels_batch.cpu().data.numpy()
        data_sc.extend(predicted)
        data_rc.extend(labels_batch)
#         if crivice == 2:
#             break
#         break
#     print(data_rc)
    fscore = f1_score(data_rc, data_sc, average = 'weighted')
    precision = precision_score(data_rc, data_sc, average = 'weighted')
    recall = recall_score(data_rc, data_sc, average = 'weighted')
    kappa = cohen_kappa_score(data_rc, data_sc)
    C = classification_report(data_rc, data_sc)
    matrix = confusion_matrix(data_rc, data_sc)
    accuracy_class = matrix.diagonal()/matrix.sum(axis=1)
    
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("Report: ", C)
    print("F1: ", fscore)
    print("Kappa: ", kappa)
    print("Matrix: ", matrix)
    print("Accuracy Class: ", accuracy_class)

    acc = 100. * correct / total
    logging.info("- Eval metrics, acc:{acc:.4f}, loss: {loss:.4f}".format(acc=acc, loss=loss))
    my_metric = {'accuracy': acc, 'loss': loss}
    #my_metric['accuracy'] = acc
    return my_metric

time: 0 ns (started: 2024-05-06 06:46:57 +06:00)


In [11]:
def train_and_evaluate_kd(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn_kd, warmup_scheduler):
    """
    KD Train the model and evaluate every epoch.
    """
    # reload weights from restore_file if specified
#     if restore_file is not None:
#         restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
#         logging.info("Restoring parameters from {}".format(restore_path))
#         load_checkpoint(restore_path, model, optimizer)

    # tensorboard setting
    log_dir = './tensorboard/'
    writer = SummaryWriter(log_dir=log_dir)
    best_val_acc = 0.0
#     teacher_model.eval()
#     teacher_acc = evaluate_kd(teacher_model, val_dataloader, params)
#     print(">>>>>>>>>The teacher accuracy: {}>>>>>>>>>".format(teacher_acc['accuracy']))
    model =model.to(device)
    scheduler = MultiStepLR(optimizer, milestones=[30, 60, 80], gamma=0.2)
    for epoch in range(100):
        if epoch > 0:   # 0 is the warm up epoch
            scheduler.step()
        logging.info("Epoch {}/{}, lr:{}".format(epoch + 1, 100, optimizer.param_groups[0]['lr']))
        print(epoch)
        # KD Train
        train_acc, train_loss = train_kd(model, optimizer, loss_fn_kd, train_dataloader, warmup_scheduler, epoch)
        # Evaluate
        val_metrics = evaluate_kd(model, val_dataloader)
#         print("val_metrics: ",val_metrics)
        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc

        # Save weights
        save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                               is_best=is_best,
                               checkpoint=".\\")
        print("Val Accuracy: ", val_acc)
        # If best_eval, best_save_path
        if is_best:
            logging.info("*********** Hurray ! Found new best accuracy *****************")
            best_val_acc = val_acc
            print("*********** Hurray ! Found new best accuracy *****************")
            print("Val Accuracy: ", val_acc)

            # Save best val metrics in a json file in the model directory
            file_name = "eval_best_result.json"
            best_json_path = os.path.join(".\\", file_name)
            save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(".\\", "eval_last_result.json")
        save_dict_to_json(val_metrics, last_json_path)

        # Tensorboard
        writer.add_scalar('Train_accuracy', train_acc, epoch)
        writer.add_scalar('Train_loss', train_loss, epoch)
        writer.add_scalar('Test_accuracy', val_metrics['accuracy'], epoch)
        writer.add_scalar('Test_loss', val_metrics['loss'], epoch)
#         export scalar data to JSON for external processing
    writer.close()


# Defining train_kd functions
def train_kd(model, optimizer, loss_fn_kd, dataloader, warmup_scheduler, epoch):
    """
    KD Train the model on `num_steps` batches
    """
    # set model to training mode
    model.train()
#     teacher_model.eval()
    loss_avg = RunningAverage()
    losses = AverageMeter()
    total = 0
    correct = 0
    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            if epoch<=0:
                warmup_scheduler.step()

            train_batch, labels_batch = train_batch.to(device), labels_batch.to(device)
            # convert to torch Variables
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # compute model output, fetch teacher output, and compute KD loss
            output_batch = model(train_batch)

            # get one batch output from teacher model
#             output_teacher_batch = teacher_model(train_batch).to(device)
#             output_teacher_batch = Variable(output_teacher_batch, requires_grad=False)

            loss = loss_fn_kd(output_batch, labels_batch)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            _, predicted = output_batch.max(1)
            total += labels_batch.size(0)
            correct += predicted.eq(labels_batch).sum().item()
            # update the average loss
            loss_avg.update(loss.data)
            losses.update(loss.item(), train_batch.size(0))

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
            t.update()

    acc = 100.*correct/total
    logging.info("- Train accuracy: {acc:.4f}, training loss: {loss:.4f}".format(acc = acc, loss = losses.avg))
    return acc, losses.avg

time: 0 ns (started: 2024-05-06 06:46:57 +06:00)


In [14]:
model = torchvision.models.resnet18(weights = "ResNet18_Weights.IMAGENET1K_V1")
model.fc = torch.nn.Linear(512, 16)

time: 188 ms (started: 2024-05-06 06:47:10 +06:00)


In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

time: 0 ns (started: 2024-05-06 06:47:11 +06:00)


In [ ]:
import warnings
warnings.filterwarnings("ignore") 

iter_per_epoch = len(train_dataloader)
optimizer = torch.optim.SGD(model.parameters(), lr= 0.1 * (batch_size / 128), momentum=0.9,
                                  weight_decay=5e-4)
warmup_scheduler = WarmUpLR(optimizer,iter_per_epoch *1)  # warmup the learning rate in the first epoch
loss = loss_kd_regularization
train_and_evaluate_kd(model, train_dataloader, val_dataloader, optimizer, loss, warmup_scheduler)

0


100%|█████████████████████████████████████████████████████| 1393/1393 [05:06<00:00,  4.54it/s, loss=0.090, lr=0.100000]


Precision:  0.8146182076315341
Recall:  0.8054674685620558
Report:                precision    recall  f1-score   support

           0       0.95      0.78      0.86      4465
           1       0.77      0.66      0.71      1945
           2       0.00      0.00      0.00        93
           3       0.71      0.89      0.79      2384
           4       0.28      0.65      0.40       170
           5       0.79      0.96      0.87      7101
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00        55
           8       0.24      0.07      0.11       179
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00        63
          11       0.00      0.00      0.00        46
          12       0.98      0.48      0.65      1780

    accuracy                           0.81     18290
   macro avg       0.36      0.35      0.34     18290
weighted avg       0.81      0.81      0.79     18290

F1:  0.7938

100%|█████████████████████████████████████████████████████| 1393/1393 [05:07<00:00,  4.52it/s, loss=0.078, lr=0.100000]


Precision:  0.7886068664822813
Recall:  0.7661016949152543
Report:                precision    recall  f1-score   support

           0       0.97      0.65      0.78      4465
           1       0.61      0.57      0.59      1945
           2       0.30      0.14      0.19        93
           3       0.60      0.94      0.73      2384
           4       0.55      0.44      0.49       170
           5       0.85      0.89      0.87      7101
           6       0.00      0.00      0.00         3
           7       1.00      0.02      0.04        55
           8       0.20      0.26      0.23       179
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00        63
          11       0.20      0.02      0.04        46
          12       0.71      0.72      0.71      1780

    accuracy                           0.77     18290
   macro avg       0.46      0.36      0.36     18290
weighted avg       0.79      0.77      0.76     18290

F1:  0.7627

...

100%|█████████████████████████████████████████████████████| 1393/1393 [05:04<00:00,  4.57it/s, loss=0.074, lr=0.020000]


Precision:  0.8389013387469404
Recall:  0.8359759431383269
Report:                precision    recall  f1-score   support

           0       0.83      0.97      0.90      4465
           1       0.70      0.83      0.76      1945
           2       0.72      0.30      0.42        93
           3       0.82      0.76      0.79      2384
           4       0.40      0.76      0.53       170
           5       0.89      0.89      0.89      7101
           6       0.00      0.00      0.00         3
           7       0.52      0.25      0.34        55
           8       0.64      0.15      0.24       179
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00        63
          11       0.00      0.00      0.00        46
          12       0.95      0.56      0.71      1780

    accuracy                           0.84     18290
   macro avg       0.50      0.42      0.43     18290
weighted avg       0.84      0.84      0.83     18290

F1:  0.8281

100%|█████████████████████████████████████████████████████| 1393/1393 [05:04<00:00,  4.57it/s, loss=0.074, lr=0.020000]


Precision:  0.858342472838464
Recall:  0.855385456533625
Report:                precision    recall  f1-score   support

           0       0.90      0.97      0.93      4465
           1       0.73      0.80      0.76      1945
           2       0.82      0.48      0.61        93
           3       0.77      0.80      0.79      2384
           4       0.30      0.75      0.43       170
           5       0.94      0.89      0.92      7101
           6       0.00      0.00      0.00         3
           7       0.69      0.20      0.31        55
           8       0.55      0.34      0.42       179
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00        63
          11       0.00      0.00      0.00        46
          12       0.84      0.73      0.78      1780

    accuracy                           0.86     18290
   macro avg       0.50      0.46      0.46     18290
weighted avg       0.86      0.86      0.85     18290

F1:  0.853936

 58%|███████████████████████████████▍                      | 810/1393 [02:58<01:54,  5.08it/s, loss=0.074, lr=0.020000]

In [23]:
def create_model():
    model = torchvision.models.resnet18(weights = "ResNet18_Weights.IMAGENET1K_V1")
    model.fc = torch.nn.Linear(512, 16)
    checkpoint = torch.load(".\\classifier\\best88.pth.tar")
    checkpoint['state_dict']
    model.load_state_dict(checkpoint['state_dict'])
    # Update the last FC layer for Tiny ImageNet number of classes.
    return model

time: 0 ns (started: 2024-05-06 06:48:45 +06:00)


In [24]:
model = create_model()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

time: 329 ms (started: 2024-05-06 06:48:46 +06:00)


In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (MB):', os.path.getsize("temp_delme.p")/1e6)
    os.remove('temp_delme.p')
print(count_parameters(model))
print(print_size_of_model(model))

11184720
Size (MB): 44.816376
None
time: 47 ms (started: 2024-05-06 06:48:48 +06:00)


In [26]:
model.to('cuda')
evaluate_kd(model, val_dataloader)

Precision:  0.9008639945306363
Recall:  0.9041006014215418
Report:                precision    recall  f1-score   support

           0       0.95      0.96      0.96      4465
           1       0.87      0.82      0.84      1945
           2       0.81      0.45      0.58        93
           3       0.84      0.89      0.86      2384
           4       0.62      0.73      0.67       170
           5       0.93      0.97      0.95      7101
           6       0.00      0.00      0.00         3
           7       0.61      0.49      0.55        55
           8       0.71      0.38      0.49       179
           9       0.00      0.00      0.00         6
          10       0.23      0.11      0.15        63
          11       0.28      0.20      0.23        46
          12       0.90      0.77      0.83      1780

    accuracy                           0.90     18290
   macro avg       0.59      0.52      0.55     18290
weighted avg       0.90      0.90      0.90     18290

F1:  0.9008

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 90.41006014215418, 'loss': 0.0}

time: 14.1 s (started: 2024-05-06 06:48:49 +06:00)


In [28]:
import nncf
import openvino as ov

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, openvino
time: 750 ms (started: 2024-05-06 07:05:00 +06:00)


In [19]:
def transform_fn(data_item):
    images, _ = data_item
    return images

calibration_dataset = nncf.Dataset(train_dataloader, transform_fn)

time: 0 ns (started: 2024-05-05 17:36:52 +06:00)


In [20]:
model.to('cpu')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

time: 32 ms (started: 2024-05-05 17:36:52 +06:00)


In [21]:
quantized_model = nncf.quantize(model, calibration_dataset)

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:nncf:Compiling and loading torch extension: quantized_functions_cpu...
Reason: Command '['where', 'cl']' returned non-zero exit status 1.
INFO:nncf:Finished loading torch extension: quantized_functions_cpu


D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\torch\utils\cpp_extension.py:384: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')


time: 4min 20s (started: 2024-05-05 17:36:52 +06:00)


In [22]:
quantized_model

ResNet(
  (conv1): NNCFConv2d(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    (pre_ops): ModuleDict(
      (0): UpdateWeight(
        (op): SymmetricQuantizer(bit=8, ch=True)
      )
    )
    (post_ops): ModuleDict()
  )
  (bn1): NNCFBatchNorm2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (pre_ops): ModuleDict()
    (post_ops): ModuleDict()
  )
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): NNCFConv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (pre_ops): ModuleDict(
          (0): UpdateWeight(
            (op): SymmetricQuantizer(bit=8, ch=True)
          )
        )
        (post_ops): ModuleDict()
      )
      (bn1): NNCFBatchNorm2d(
        64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (pre_ops): ModuleDict()
        (p

time: 0 ns (started: 2024-05-05 17:46:06 +06:00)


In [23]:
quantized_model.to('cpu')
evaluate_kd(quantized_model, val_dataloader, device = 'cpu')

Precision:  0.9055983207731232
Recall:  0.9110442864953526
Report:                precision    recall  f1-score   support

           0       0.95      0.97      0.96      4465
           1       0.88      0.84      0.86      1945
           2       0.83      0.38      0.52        93
           3       0.86      0.89      0.87      2384
           4       0.64      0.84      0.72       170
           5       0.93      0.97      0.95      7101
           6       0.00      0.00      0.00         3
           7       0.90      0.47      0.62        55
           8       0.66      0.41      0.51       179
           9       0.00      0.00      0.00         6
          10       0.17      0.03      0.05        63
          11       0.00      0.00      0.00        46
          12       0.91      0.80      0.86      1780

    accuracy                           0.91     18290
   macro avg       0.59      0.51      0.53     18290
weighted avg       0.91      0.91      0.91     18290

F1:  0.9065

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 91.10442864953527, 'loss': 0.0}

time: 2min 24s (started: 2024-05-05 17:46:10 +06:00)


In [24]:
print(count_parameters(quantized_model))
print(print_size_of_model(quantized_model))

11189566
Size (MB): 44.923096
None
time: 125 ms (started: 2024-05-05 17:48:34 +06:00)


In [25]:
dummy_input = torch.randn(128, 3, 64,64)

quantized_model_ir = ov.convert_model(quantized_model, example_input=dummy_input, input=[-1, 3, 64,64])

ov.save_model(quantized_model_ir, ".\\classifier\\quantized_model_focal.xml")

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\nncf\torch\quantization\layers.py:337: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return self._level_low.item()
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\nncf\torch\quantization\layers.py:345: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return self._level_high.item()
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\torch\jit\_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not c

time: 8.91 s (started: 2024-05-05 17:48:34 +06:00)


In [31]:
from openvino.runtime.ie_api import CompiledModel
from typing import Union


def validate(model, val_loader):
    """Compute the metrics using data from val_loader for the model"""
    total = 0
    correct = 0
    data_sc = []
    data_rc = []
    start_time = time.time()
    # Switch to evaluate mode.
    if not isinstance(model, CompiledModel):
        model.eval()
        model.to(torch_device)

    with torch.no_grad():
        end = time.time()
        for i, (images, labels_batch) in enumerate(val_loader):
            images = images.to('cpu')
            labels_batch = labels_batch.to('cpu')

            # Compute the output.
            if isinstance(model, CompiledModel):
                output_layer = model.output(0)
                output = model(images)[output_layer]
                output_batch = torch.from_numpy(output)
#             print(output)
            loss = 0.0  # force validation loss to zero to reduce computation time
            _, predicted = output_batch.max(1)
            total += labels_batch.size(0)
            correct += predicted.eq(labels_batch).sum().item()
            predicted = predicted.cpu().data.numpy()
            labels_batch = labels_batch.cpu().data.numpy()
            data_sc.extend(predicted)
            data_rc.extend(labels_batch)

    #         break
    #     print(data_rc)
        fscore = f1_score(data_rc, data_sc, average = 'weighted')
        precision = precision_score(data_rc, data_sc, average = 'weighted')
        recall = recall_score(data_rc, data_sc, average = 'weighted')
        kappa = cohen_kappa_score(data_rc, data_sc)
        C = classification_report(data_rc, data_sc)
        matrix = confusion_matrix(data_rc, data_sc)
        accuracy_class = matrix.diagonal()/matrix.sum(axis=1)

        print("Precision: ", precision)
        print("Recall: ", recall)
        print("Report: ", C)
        print("F1: ", fscore)
        print("Kappa: ", kappa)
        print("Matrix: ", matrix)
        print("Accuracy Class: ", accuracy_class)

        acc = 100. * correct / total
        logging.info("- Eval metrics, acc:{acc:.4f}, loss: {loss:.4f}".format(acc=acc, loss=loss))
        my_metric = {'accuracy': acc, 'loss': loss}
        #my_metric['accuracy'] = acc
        return my_metric

time: 0 ns (started: 2024-05-06 07:05:34 +06:00)


In [27]:
core = ov.Core()
int8_compiled_model = core.compile_model(quantized_model_ir, 'CPU')
validate(int8_compiled_model, val_dataloader )

Precision:  0.9053496938631382
Recall:  0.9108255877528704
Report:                precision    recall  f1-score   support

           0       0.95      0.96      0.96      4465
           1       0.88      0.84      0.86      1945
           2       0.82      0.39      0.53        93
           3       0.86      0.89      0.88      2384
           4       0.64      0.83      0.72       170
           5       0.93      0.97      0.95      7101
           6       0.00      0.00      0.00         3
           7       0.90      0.47      0.62        55
           8       0.66      0.41      0.51       179
           9       0.00      0.00      0.00         6
          10       0.17      0.03      0.05        63
          11       0.00      0.00      0.00        46
          12       0.91      0.80      0.86      1780

    accuracy                           0.91     18290
   macro avg       0.59      0.51      0.53     18290
weighted avg       0.91      0.91      0.91     18290

F1:  0.9062

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 91.08255877528704, 'loss': 0.0}

time: 13.1 s (started: 2024-05-05 17:48:43 +06:00)


In [29]:
int8_ir_path = ".\\classifier\\quantized_model_2.xml"
print(f"[2/7] Save INT8 model: {int8_ir_path}")

core = ov.Core()

model_q = core.read_model(model=int8_ir_path)
model_q = core.compile_model(model=model_q, device_name='CPU')

[2/7] Save INT8 model: .\classifier\quantized_model_2.xml
time: 297 ms (started: 2024-05-06 07:05:18 +06:00)


In [32]:
validate(model_q, val_dataloader )

Precision:  0.8763404612825163
Recall:  0.8769819573537452
Report:                precision    recall  f1-score   support

           0       0.95      0.87      0.91      4465
           1       0.86      0.81      0.83      1945
           2       0.36      0.37      0.36        93
           3       0.74      0.91      0.82      2384
           4       0.65      0.72      0.68       170
           5       0.92      0.96      0.94      7101
           6       0.00      0.00      0.00         3
           7       0.73      0.40      0.52        55
           8       0.56      0.36      0.44       179
           9       0.00      0.00      0.00         6
          10       0.27      0.11      0.16        63
          11       0.00      0.00      0.00        46
          12       0.85      0.77      0.80      1780

    accuracy                           0.88     18290
   macro avg       0.53      0.48      0.50     18290
weighted avg       0.88      0.88      0.87     18290

F1:  0.8743

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 87.69819573537453, 'loss': 0.0}

time: 12.4 s (started: 2024-05-06 07:05:39 +06:00)


In [ ]:
# class QuantizedResNet18(torch.nn.Module):
#     def __init__(self, model_fp32):
#         super(QuantizedResNet18, self).__init__()
  
#         self.quant = torch.quantization.QuantStub()
#         # FP32 model 
#         self.model_fp32 = model_fp32
#         self.dequant = torch.quantization.DeQuantStub()


#     def forward(self, x):
#         x = self.quant(x)
#         x = self.model_fp32(x)
#         x = self.dequant(x)
#         return x

In [ ]:
# model_q = QuantizedResNet18(model)

In [ ]:
# model_q.eval()
# model_q.qconfig = torch.ao.quantization.default_qconfig
# model_q = torch.ao.quantization.prepare(model_q)
# model_q

In [ ]:
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000+
    